In [1]:
!python --version

Python 3.11.7


In [4]:
!pip install --upgrade google-cloud-aiplatform

  Using cached google_cloud_aiplatform-1.39.0-py2.py3-none-any.whl.metadata (28 kB)
  Using cached google_api_core-2.15.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_cloud_storage-2.14.0-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached google_cloud_bigquery-3.16.0-py2.py3-none-any.whl.metadata (8.8 kB)
  Using cached google_cloud_resource_manager-1.11.0-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached shapely-2.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (7.0 kB)
  Using cached googleapis_common_protos-1.62.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached google_auth-2.26.2-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached grpcio-1.60.0-cp311-cp311-macosx_10_10_universal2.whl.metadata (4.0 kB)
  Using cached grpcio_status-1.60.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.0

In [8]:
!pip install Pillow

#### Main Script

In [30]:
from PIL import Image
import os
import base64
from io import BytesIO
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = Image.open(os.path.join(folder_path, filename))
            images.append((filename, img))
    return images

def convert_to_base64(images):
    base64_images = []
    for filename, img in images:
        buffered = BytesIO()
        img.save(buffered, format="PNG")  # Change format to "JPEG" if needed
        img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        base64_images.append({'name': filename, 'base64': img_base64})
    return base64_images

def generate(image):
  combined_response = ""
  image_decoded = Part.from_data(data=base64.b64decode(image), mime_type="image/jpeg")
  model = GenerativeModel("gemini-pro-vision")
  responses = model.generate_content(
    [image_decoded, """Read the text in this image."""],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
  stream=True,
  )
  
  for response in responses:
      #print(response.text, end="")
      combined_response += response.text
  return combined_response

def save_text_to_files(text_for_images, predefined_folder):
    """
    Save Base64 representations of images to text files.

    Parameters:
    base64_images (list of dict): List containing dictionaries with image names and Base64 strings.
    predefined_folder (str): Path to the folder where text files should be saved.
    """
    # Create the folder if it doesn't exist
    if not os.path.exists(predefined_folder):
        os.makedirs(predefined_folder)

    for item in text_for_images:
        # Extract the image name (without extension) and Base64 string
        image_name = os.path.splitext(item['name'])[0]
        base64_string = item['text']

        # Construct the full path for the text file
        file_path = os.path.join(predefined_folder, image_name + '.txt')

        # Write the Base64 string to the file
        with open(file_path, 'w') as file:
            file.write(base64_string)

    print("handwritten text has been saved to text files.")

vertexai.init(project="project-name")
input_folder_path = 'path_to_images_with_handwritten_text'
output_folder_path = 'path_where_text_files_will_be_generated'
images = read_images_from_folder(input_folder_path)
base64_images = convert_to_base64(images)

text_for_images = []
for image in base64_images:
    print(image['name']);
    #print(image['base64']);
    response = generate(image['base64'])
    text_for_images.append({'name': image['name'], 'text': response})

save_text_to_files(text_for_images, output_folder_path)


Lettre_de_Dickens_à_Angela_Burdett-Coutts.jpg
autograph-letter-from-charles-dickens-written-to-charles-kent-on-the-ERH7A3.jpg
anotherLetter.jpg
charles-dickens-signed-handwritten-letter-from-the-english-novelist-ERHEW1.jpg
handwritten text has been saved to text files.
